In [28]:
import numpy as np
import pandas as pd
import epo_ops
import xml.etree.ElementTree as ET

In [29]:
with open('keys/epo_api_key', 'r') as file:
    consumer_secret = file.read().strip()
consumer_key = "5wpmow69fw4pjAXRrSKKVrlpHpuGgX9bFwlrVmX9XGAKKCHo"

In [30]:
names = ["FW", "LC", "LIB",  "SC", "SPV", "Wind"]  # technology file names
# names = ["Wind"]  # technology file names"Wind"

In [31]:
all_dfs = []
for n in names:
    df = pd.read_excel(f'unlabeled_data/{n}_EPO.xlsx')
    all_dfs.append(df)

In [32]:
def raw_to_prop_pub(pub_nrs: list):
    for i in range(len(pub_nrs)):
        diff = 7 - len(pub_nrs[i])
        if diff > 0:
            pub_nrs[i] = '0' * diff + pub_nrs[i]
        elif diff < 0:
            print('invalid publication number')
            pub_nrs[i] = np.nan
        pub_nrs[i] = 'EP' + pub_nrs[i]
    return pub_nrs

In [33]:
def resp_to_pub_date(response):
  # Parse XML
  xml_response = response.text

  ns = {"epo": "http://www.epo.org/exchange"}  # Define the namespace
  root = ET.fromstring(xml_response)

  # Find the publication date using XPath
  pub_date = root.find(".//epo:publication-reference/epo:document-id/epo:date", ns)

  if pub_date is not None:
    dt = pd.to_datetime(pub_date.text[:8], format='%Y%m%d')
    # print(f"Publication Year: {dt.year}")
    return dt
  else:
    # print("Publication date not found.")
    return None

In [34]:
def get_EP_pub_date(pub_numbers):
  """
  runs API to get publication date in format YYYYMMDD
  """
  pub_date = []
  client = epo_ops.Client(key=consumer_key, secret=consumer_secret)  # Instantiate client
  for p in pub_numbers:
    try:
      response = client.published_data(  # Retrieve bibliography data
        reference_type = 'publication',  # publication, application, priority
        input = epo_ops.models.Epodoc(p), # 
        endpoint = 'biblio',  # optional, defaults to biblio in case of published_data
        constituents = []  # optional, list of constituents
      )
      dt = resp_to_pub_date(response)
      if dt is not None:
        pub_date.append(dt)
      else:
        pub_date.append(np.nan)
    except Exception as e:
      print('client call failed: ')
      print(e)
      pub_date.append(np.nan)
      continue
  return pub_date

In [ ]:
i=0
for df in all_dfs:
    print('starting date extraction for', names[i])
    
    df.dropna(subset=['Publn_nr'], inplace=True)    # drop non defined publication numbers
    
    pub_numbers = df['Publn_nr'].unique().astype(list)
    df_u = df.drop_duplicates(subset=['Publn_nr'])  # unique claims (sort by patent)
    pub_numbers = pub_numbers.astype(int).astype(str)

    pub_formatted = raw_to_prop_pub(pub_numbers)   # convert to proper format

    # run the API request
    list_of_dfs = []
    dates = get_EP_pub_date(pub_formatted)

    # save dates
    df_dates = pd.DataFrame({'Publn_nr': pub_formatted, 'pub_date': dates})
    df_dates.to_csv(f'unlabeled_data/{names[i]}_EPO_dates.csv', index=False)
    i += 1

starting date extraction for FW
